# Loading python libraries
#### Python is developed under an OSI-approved open source license, making it freely usable and distributable, even for commercial use.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import os
import glob
from sklearn import preprocessing
from pysptools import spectro as sp
from pysptools import distance
from IPython.display import display
import ipywidgets as widgets
from math import*
from ipywidgets import IntProgress
import peakutils
import rampy as rp

# Loading Functions

* Widegets
    * on_value_change_min - this function monitors the change in value of the slider
    * on_value_change_max - this function monitors the change in value of the slider
* FolderAndFileProcessing
    * ReadFilePaths - This function takes input as source folder and file extension and returns the path of all the files inside the folder with the supplied extension
    * ReadFile - This functions purpose is to read individual file, it takes file path, rows to skip while reading the file and type of seperation as input
    * GetValuesFromFile - This function uses ReadFile read a file and then filters out desired column
    * WriteDictToExcel - This function is used to write the output of the analysis to excel file
    * combine_dicts - This function is used to combine two dictionaries in python into one dictionary
* SpectraComparision(all functions normalizes and removes continum after subsetting the spectral signals)
    * SFF - This function takes reference spectra and user input spectrum as input and process the signals using root mean squared algorithm and generates the score table with most probable mineral having the least score
    * SAM - This function takes reference spectra and user input spectrum as input and process the signals using root mean squared algorithm and generates the score table with most probable mineral having the least score
    * SFF_CrossCorrelation - This function takes reference spectra and user input spectrum as input and process the signals using cross-correlation algorithm and generates the score table with most probable mineral having the highest score(closer to 1) and least probable mineral having the lowest score(closer to -1)
    * SpectralInformationDivergence - Computes the spectral information divergence between two vectors.

In [2]:
### https://pysptools.sourceforge.io/distance.html
### https://dataaspirant.com/five-most-popular-similarity-measures-implementation-in-python/
### https://towardsdatascience.com/four-ways-to-quantify-synchrony-between-time-series-data-b99136c4a9c9
### https://towardsdatascience.com/classical-least-squares-method-for-quantitative-spectral-analysis-with-python-1926473a802c

class Widegets:
    def on_value_change_min(change):
        new_min = change['new']
        if new_min + 100 > maxWavelength.value:
            minWavelength.value = maxWavelength.value - 100

    def on_value_change_max(change):
        new_max = change['new']
        if minWavelength.value > new_max - 100:
            maxWavelength.value = minWavelength.value + 100
        
        
class FolderAndFileProcessing:
    def ReadFilePaths(source, ext):
        return glob.glob(source + "/**/*." + ext, recursive = True)
    
    def ReadFile(file, seperation):
        return pd.read_csv(file, sep = seperation)
    
    def GetValuesFromFile(file, seperation, column):
        fileData = FolderAndFileProcessing.ReadFile(file, seperation)
        return fileData[column]
    
    def WriteDictToExcel(targetPath, filename, dictionary):
        df = pd.DataFrame.from_dict(dictionary)
        os.makedirs(targetPath, exist_ok=True)
        df.to_excel(targetPath+filename)
        
        
    def combine_dicts(a, b):
        for key in b:
            if key in a:
                b[key] = b[key] + a[key]
        c = {**a, **b}
        return c
        
class SpectraComparision:
    
    def SFF(fileList, spectraValue, fileListBaselineRemoval, waveLength, minIndex, maxIndex):
        mineralScore = {}
        normalized_spectraValue = preprocessing.normalize([spectraValue[minIndex:maxIndex]])[0]
        try:
            baseline_values = peakutils.baseline(normalized_spectraValue)
            baselineRemoved_spectraValue = normalized_spectraValue - baseline_values
            #continumRemoved_spectraValue = sp.convex_hull_removal(normalized_spectraValue[minIndex:maxIndex], waveLength[minIndex:maxIndex])[0]
        except:
            return mineralScore
        for index in range(len(fileList)):
            baselineRemoved_idealReflectance = fileListBaselineRemoval[index]
            result = np.array(baselineRemoved_spectraValue) - np.array(baselineRemoved_idealReflectance)
            score = np.sqrt(np.mean(result**2))
            score = 1 - score
            mineralScore[fileList[index].split('/')[-1].split('_')[0]] = score
        mineralScore = sorted(mineralScore.items(), key=lambda x: x[1], reverse=True)
        return mineralScore
    
    def SFF_CrossCorrelation(fileList, spectraValue, fileListBaselineRemoval, waveLength, minIndex, maxIndex):
        mineralScore = {}
        normalized_spectraValue = preprocessing.normalize([spectraValue[minIndex:maxIndex]])[0]
        try:
            baseline_values = peakutils.baseline(normalized_spectraValue)
            baselineRemoved_spectraValue = normalized_spectraValue - baseline_values
            #continumRemoved_spectraValue = sp.convex_hull_removal(normalized_spectraValue[minIndex:maxIndex], waveLength[minIndex:maxIndex])[0]
        except:
            return mineralScore
        for index in range(len(fileList)):
            baselineRemoved_idealReflectance = fileListBaselineRemoval[index]
            result = np.corrcoef(baselineRemoved_spectraValue, baselineRemoved_idealReflectance)
            score = result[0, 1]
            mineralScore[fileList[index].split('/')[-1].split('_')[0]] = score
        mineralScore = sorted(mineralScore.items(), key=lambda x: x[1], reverse=True)
        return mineralScore

    
    def CosineSimilarity(fileList, spectraValue, fileListBaselineRemoval, waveLength, minIndex, maxIndex):
        mineralScore = {}
        normalized_spectraValue = preprocessing.normalize([spectraValue[minIndex:maxIndex]])[0]
        try:
            baseline_values = peakutils.baseline(normalized_spectraValue)
            baselineRemoved_spectraValue = normalized_spectraValue - baseline_values
            #continumRemoved_spectraValue = sp.convex_hull_removal(normalized_spectraValue[minIndex:maxIndex], waveLength[minIndex:maxIndex])[0]
        except:
            return mineralScore
        for index in range(len(fileList)):
            baselineRemoved_idealReflectance = fileListBaselineRemoval[index]
            score = SpectraComparision.cosine_similarity(baselineRemoved_spectraValue, baselineRemoved_idealReflectance)
            score = 1 - score
            mineralScore[fileList[index].split('/')[-1].split('_')[0]] = score
        mineralScore = sorted(mineralScore.items(), key=lambda x: x[1], reverse=True)
        return mineralScore
    
    
    
    def computeSimilarity(fileList, spectraValue, fileListBaselineRemoval, waveLength, minIndex, maxIndex):
        mineralScore = {}
        normalized_spectraValue = preprocessing.normalize([spectraValue[minIndex:maxIndex]])[0]
        try:
            baseline_values = peakutils.baseline(normalized_spectraValue)
            baselineRemoved_spectraValue = normalized_spectraValue - baseline_values
            #continumRemoved_spectraValue = sp.convex_hull_removal(normalized_spectraValue[minIndex:maxIndex], waveLength[minIndex:maxIndex])[0]
        except:
            return mineralScore
        for index in range(len(fileList)):
            baselineRemoved_idealReflectance = fileListBaselineRemoval[index]
            score = SpectraComparision.compute_similarity(baselineRemoved_spectraValue, baselineRemoved_idealReflectance)
            score = 1 - score
            mineralScore[fileList[index].split('/')[-1].split('_')[0]] = score
        mineralScore = sorted(mineralScore.items(), key=lambda x: x[1], reverse=True)
        return mineralScore
    
    def SAM(fileList, spectraValue, fileListBaselineRemoval, waveLength, minIndex, maxIndex):
        mineralScore = {}
        normalized_spectraValue = preprocessing.normalize([spectraValue[minIndex:maxIndex]])[0]
        try:
            baseline_values = peakutils.baseline(normalized_spectraValue)
            baselineRemoved_spectraValue = normalized_spectraValue - baseline_values
            #continumRemoved_spectraValue = sp.convex_hull_removal(normalized_spectraValue[minIndex:maxIndex], waveLength[minIndex:maxIndex])[0]
        except:
            return mineralScore
        for index in range(len(fileList)):
            baselineRemoved_idealReflectance = fileListBaselineRemoval[index]
            score = distance.SAM(baselineRemoved_spectraValue, baselineRemoved_idealReflectance)
            score = 1 - score
            mineralScore[fileList[index].split('/')[-1].split('_')[0]] = score
        mineralScore = sorted(mineralScore.items(), key=lambda x: x[1], reverse=True)
        return mineralScore
    
    def chebyshev(fileList, spectraValue, fileListBaselineRemoval, waveLength, minIndex, maxIndex):
        mineralScore = {}
        normalized_spectraValue = preprocessing.normalize([spectraValue[minIndex:maxIndex]])[0]
        try:
            baseline_values = peakutils.baseline(normalized_spectraValue)
            baselineRemoved_spectraValue = normalized_spectraValue - baseline_values
            #continumRemoved_spectraValue = sp.convex_hull_removal(normalized_spectraValue[minIndex:maxIndex], waveLength[minIndex:maxIndex])[0]
        except:
            return mineralScore
        for index in range(len(fileList)):
            baselineRemoved_idealReflectance = fileListBaselineRemoval[index]
            score = distance.chebyshev(baselineRemoved_spectraValue, baselineRemoved_idealReflectance)
            mineralScore[fileList[index].split('/')[-1].split('_')[0]] = score
        mineralScore = sorted(mineralScore.items(), key=lambda x: x[1], reverse=True)
        return mineralScore
    
    def SpectralInformationDivergence(fileList, spectraValue, fileListBaselineRemoval, waveLength, minIndex, maxIndex):
        mineralScore = {}
        normalized_spectraValue = preprocessing.normalize([spectraValue[minIndex:maxIndex]])[0]
        try:
            baseline_values = peakutils.baseline(normalized_spectraValue)
            baselineRemoved_spectraValue = normalized_spectraValue - baseline_values
            #continumRemoved_spectraValue = sp.convex_hull_removal(normalized_spectraValue[minIndex:maxIndex], waveLength[minIndex:maxIndex])[0]
        except:
            return mineralScore
        for index in range(len(fileList)):
            baselineRemoved_idealReflectance = fileListBaselineRemoval[index]
            score = distance.SID(baselineRemoved_spectraValue, baselineRemoved_idealReflectance)
            score = 1 - score
            mineralScore[fileList[index].split('/')[-1].split('_')[0]] = score
        mineralScore = sorted(mineralScore.items(), key=lambda x: x[1], reverse=True)
        return mineralScore

    
    def compute_similarity(ref_rec,input_rec,weightage=[0.33,0.33,0.33]):
        ref_rec = np.array(ref_rec)
        input_rec = np.array(input_rec)
        ## Time domain similarity
        ref_time = np.correlate(ref_rec,ref_rec)    
        inp_time = np.correlate(ref_rec,input_rec)
        diff_time = abs(ref_time-inp_time)

        ## Freq domain similarity
        ref_freq = np.correlate(np.fft.fft(ref_rec),np.fft.fft(ref_rec)) 
        inp_freq = np.correlate(np.fft.fft(ref_rec),np.fft.fft(input_rec))
        diff_freq = abs(ref_freq-inp_freq)

        ## Power similarity
        ref_power = np.sum(ref_rec**2)
        inp_power = np.sum(input_rec**2)
        diff_power = abs(ref_power-inp_power)
        
        return float(weightage[0]*diff_time+weightage[1]*diff_freq+weightage[2]*diff_power)
    

    
    def square_rooted(x):
        return round(sqrt(sum([a*a for a in x])),3)
 
    def cosine_similarity(x,y):
        numerator = sum(a*b for a,b in zip(x,y))
        denominator = SpectraComparision.square_rooted(x)* SpectraComparision.square_rooted(y)
        return round(numerator/float(denominator),3)

### Variable Declarations

In [3]:
##txt file declarations
txtPath = "../data/LR-Raman_Processed/"
extensionTxt= 'txt'

###sed file declarations
espPath = "../data/Vulcano Raman Processed/"
extensionEsp= 'txt'

indidualplotlimit = 10
limit = 6

minWavelength = []
maxWavelength = []

minIndex = []
maxIndex = []


numberOfFeatureToAnalyze = int(input("Enter the totlal number of features you want to anaylze: "))

Enter the totlal number of features you want to anaylze: 1


# Image Selection
### Select if you want to plot graph or not

In [4]:
toggleButton = widgets.ToggleButtons(
    options=['No Image', 'Image'],
    description='Select:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Generates only excel file', 'Generates excel files with graph plots'],
#    icons=['check'] * 3
)
#toggleButton.observe(Widegets.on_value_change_toggle, names='value')

In [6]:
display(toggleButton)

ToggleButtons(description='Select:', index=1, options=('No Image', 'Image'), tooltips=('Generates only excel f…

# Full Plot
### Select if you want to plot graph for the entire spectral range

In [7]:
toggleButtonFullPlot = widgets.ToggleButtons(
    options=['Yes', 'No'],
    description='Select:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Also Generates Full plot', 'Only generates plot of selected region'],
#    icons=['check'] * 3
)

In [9]:
display(toggleButtonFullPlot)

ToggleButtons(description='Select:', index=1, options=('Yes', 'No'), tooltips=('Also Generates Full plot', 'On…

# Individual plots
### Select if you want to plot the graph for all the minerals together or each probale output as sepererate graph plot

In [10]:
toggleButtonIndividualPlot = widgets.ToggleButtons(
    options=['Yes', 'No'],
    description='Select:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Generates Individual plots', 'Generates Combined Plots'],
#    icons=['check'] * 3
)

In [11]:
display(toggleButtonIndividualPlot)

ToggleButtons(description='Select:', options=('Yes', 'No'), tooltips=('Generates Individual plots', 'Generates…

# Declaring dynamic functions
### Creating dynamic functions so the code if more adaptable and not hardcoded

In [12]:
for i in range(numberOfFeatureToAnalyze):
    exec(f"""def on_value_change_min_{i}(change): 
         new_min = change['new']
         if new_min + 100 > maxWavelength[{i}].value:
             minWavelength[{i}].value = maxWavelength[{i}].value - 100""")
    
    exec(f"""def on_value_change_max_{i}(change): 
         new_max = change['new']
         if minWavelength[{i}].value > new_max - 100:
             maxWavelength[{i}].value = minWavelength[{i}].value + 100""")

In [13]:
###Descibing wavelengths
for ind in range(numberOfFeatureToAnalyze):
    minWavelength.append(widgets.IntSlider(value=160, min=160, max=4000))
    maxWavelength.append(widgets.IntSlider(value=4000, min=160, max=4000))
    minWavelength[ind].description = 'min wav'
    maxWavelength[ind].description = 'max wav'
        
    minWavelength[ind].observe(globals()['on_value_change_min_'+ str(ind)], names='value')
    maxWavelength[ind].observe(globals()['on_value_change_max_'+ str(ind)], names='value')

# Select the wavelength range
### Select the range to subset the data

In [15]:
for ind in range(numberOfFeatureToAnalyze):
    print('feature' + str(ind+1))
    display(minWavelength[ind], maxWavelength[ind])

feature1


IntSlider(value=800, description='min wav', max=4000, min=160)

IntSlider(value=1200, description='max wav', max=4000, min=160)

In [16]:
minIndex = []
maxIndex = []
for ind in range(numberOfFeatureToAnalyze):
    minIndex.append(minWavelength[ind].value - 160)
    maxIndex.append(maxWavelength[ind].value - 160 + 1)

In [17]:
maxIndex[0]

1041

# Populating paths
### Getting the paths to both reference spectrum and user input spectrum

In [18]:
txt_files = FolderAndFileProcessing.ReadFilePaths(txtPath,extensionTxt)
esp_files = FolderAndFileProcessing.ReadFilePaths(espPath,extensionEsp)

# Defining wavelength range

In [19]:
waveLength = list(range(160,4001))

# Continium removal after splicing the data
### Splice the ideal spectral data and remove the continium

In [20]:
max_count = len(txt_files)
for ind in range(numberOfFeatureToAnalyze):
    minIndexTemp = minIndex[ind]
    maxIndexTemp = maxIndex[ind]
    globals()['fileListBaselineRemoval_'+str(ind)] = []
    globals()['progreesBar'+str(ind)] = IntProgress(min=0, max=max_count) # instantiate the bar
    display(globals()['progreesBar'+str(ind)]) # display the bar

    for index in range(len(txt_files)):
        #print(txt_files[index])
        idealReflectance = FolderAndFileProcessing.GetValuesFromFile(txt_files[index], ' ','reflectance')
        
        
        normalized_idealReflectance = preprocessing.normalize([idealReflectance[minIndexTemp:maxIndexTemp]])[0]
        baseline_values = peakutils.baseline(normalized_idealReflectance)
        baselineRemoved_idealReflectance = normalized_idealReflectance - baseline_values
        
        
        globals()['fileListBaselineRemoval_'+str(ind)].append(baselineRemoved_idealReflectance)
        globals()['progreesBar'+str(ind)].style = {'description_width': 'initial'}
        globals()['progreesBar'+str(ind)].description = 'Loading ideal files feature '+ str(ind + 1) + ': ' + str(index + 1) + '/' + str(max_count)
        globals()['progreesBar'+str(ind)].value += 1 # signal to increment the progress bar
        
        

IntProgress(value=0, max=9554)

### Remove the continium from the entire wavelegth of ideal spectral data 

In [21]:
fileListBaselineRemoval = []
max_count = len(txt_files)

if toggleButtonFullPlot.value == 'Yes':
    progreesBar = IntProgress(min=0, max=max_count) # instantiate the bar
    display(progreesBar) # display the bar
    
    for index in range(len(txt_files)):
        idealReflectance = FolderAndFileProcessing.GetValuesFromFile(txt_files[index], ' ','reflectance')

        
        normalized_idealReflectance = preprocessing.normalize([idealReflectance])[0]
        baseline_values = peakutils.baseline(normalized_spectraValue)
        
        
        fileListContinumRemoval.append(baselineRemoved_idealReflectance)
        
        
        #counter = counter + 1
        #print('processed ' + str(counter))
        progreesBar.style = {'description_width': 'initial'}
        progreesBar.description = 'Loading ideal files ' + str(index + 1)
        progreesBar.value += 1 # signal to increment the progress bar

# Analysis
### Taking user data as input and writing the result in similar type of folder structure with weighted output for each input data stored with similar name in an excel file and graph plots stored as png

In [23]:
max_countEsp = len(esp_files)
progreesBarEsp = IntProgress(min=0, max=max_countEsp) 
display(progreesBarEsp)
targetFolderToReplace = '/Vulcano Raman Processed'
targetFolderNewName = '/Raman_SFF-Cross_800_1200_15Top'

for index in range(len(esp_files)):
    totalDict = {}
    testData = FolderAndFileProcessing.GetValuesFromFile(esp_files[index], ' ','reflectance')
    
    targetfilename = esp_files[index].replace(targetFolderToReplace, targetFolderNewName).split('/')[-1].replace('.txt','.xlsx')
    list_string = esp_files[index].replace(targetFolderToReplace, targetFolderNewName).split('/')[0:-1]
    targetPath = '/'.join(list_string) + '/'
    #testData = FolderAndFileProcessing.GetValuesFromFile(esp_files[index], 26, '\t','Reflect. [1.0]')
    minIndexTemp = 0
    maxIndexTemp = 0
    for indexRange in range(numberOfFeatureToAnalyze):
        minIndexTemp = minIndex[indexRange]
        maxIndexTemp = maxIndex[indexRange]
        dictMineralValue = SpectraComparision.SFF_CrossCorrelation(txt_files, testData, globals()['fileListBaselineRemoval_'+str(indexRange)], waveLength, minIndexTemp, maxIndexTemp)
        if len(totalDict) == 0:
            totalDict = dict(dictMineralValue)
        else:
            tempDict = dict(dictMineralValue)
            totalDict = FolderAndFileProcessing.combine_dicts(totalDict, tempDict)
    totalDict = {k: v / numberOfFeatureToAnalyze for k, v in dict(totalDict).items()}
    totalDict = sorted(totalDict.items(), key=lambda x: x[1], reverse=True)
    FolderAndFileProcessing.WriteDictToExcel(targetPath, targetfilename, totalDict)
    
    
    if toggleButton.value == 'No Image':
        continue
    
    
    if toggleButtonIndividualPlot.value == 'No':
        
        try:
            counterMineral = 0
            mineralNameList = [i for i,j in totalDict]
            mineralNameListPruned = [i.split(' ')[0] for i in mineralNameList]
            #mineralNameListDict = dict.fromkeys(set(mineralNameListPruned), False)
            
            normalized_spectraValue = preprocessing.normalize([testData[minIndexTemp:maxIndexTemp]])[0]
            
            minIndexTemp = 0
            maxIndexTemp = 0
            
            for indexRange in range(numberOfFeatureToAnalyze):
                minIndexTemp = minIndex[indexRange]
                maxIndexTemp = maxIndex[indexRange]
                
                baseline_values = peakutils.baseline(normalized_spectraValue)
                baselineRemoved_spectraValue = normalized_spectraValue - baseline_values
                
                #continumRemoved_spectraValue = sp.convex_hull_removal(normalized_spectraValue[minIndexTemp:maxIndexTemp], waveLength[minIndexTemp:maxIndexTemp])[0]
                xaxis = list(range(minWavelength[indexRange].value,maxWavelength[indexRange].value + 1))
                fig = plt.figure(figsize=(24, 12))
                plt.plot(xaxis, baselineRemoved_spectraValue, label = targetfilename.replace('.xlsx',''))
                
                indexForWhile = 0
                counterMineral = 0
                mineralNameListDict = dict.fromkeys(set(mineralNameListPruned), False)
                while counterMineral < limit:
                    mineralName = totalDict[indexForWhile][0]
                    
                    if mineralNameListDict[mineralName.split(' ')[0]] == False:
                        imagePath = list(s for s in txt_files if mineralName.lower() in s.lower())[0]
                        signal = globals()['fileListBaselineRemoval_'+str(indexRange)][txt_files.index(imagePath)]
                        plt.plot(xaxis, signal, label = mineralName)
                        mineralNameListDict[mineralName.split(' ')[0]] = True
                        counterMineral = counterMineral + 1
                    
                    indexForWhile = indexForWhile + 1
                imageLabelRange = '_' + str(minIndexTemp + 160) + '_' + str(maxIndexTemp + 160 - 1)
                plt.legend(loc="upper left")
                plt.savefig(esp_files[index].replace(targetFolderToReplace, targetFolderNewName).replace('.txt',imageLabelRange + '.png'), dpi=500)
                plt.close()
        except:
            continue
        
        if toggleButtonFullPlot.value == 'Yes':
            try:
                counterMineral = 0
                mineralNameList = [i for i,j in totalDict]
                mineralNameListPruned = [i.split(' ')[0] for i in mineralNameList]
                
                normalized_spectraValue = preprocessing.normalize([testData])[0]
                #continumRemoved_spectraValue = sp.convex_hull_removal(normalized_spectraValue, waveLength)[0]
                
                baseline_values = peakutils.baseline(normalized_spectraValue)
                baselineRemoved_spectraValue = normalized_spectraValue - baseline_values
                    
                fig = plt.figure(figsize=(24, 12))
                plt.plot(waveLength, baselineRemoved_spectraValue, label = targetfilename.replace('.xlsx',''))
                    
                indexForWhile = 0
                counterMineral = 0
                mineralNameListDict = dict.fromkeys(set(mineralNameListPruned), False)
                while counterMineral < limit:
                    mineralName = totalDict[indexForWhile][0]
                    
                    if mineralNameListDict[mineralName.split(' ')[0]] == False:
                        imagePath = list(s for s in txt_files if mineralName.lower() in s.lower())[0]
                        signal = continumRemoved_spectraValue[txt_files.index(imagePath)]
                        plt.plot(waveLength, signal, label = mineralName)
                        mineralNameListDict[mineralName.split(' ')[0]] = True
                        counterMineral = counterMineral + 1
                    
                    indexForWhile = indexForWhile + 1
                imageLabelRange = '_FullPlot'
                plt.legend(loc="upper left")
                plt.savefig(esp_files[index].replace(targetFolderToReplace, targetFolderNewName).replace('.txt',imageLabelRange + '.png'), dpi=500)
                plt.close()
            except:
                continue
            
            
            
            
            
            
        
     
    
    
    else:
        try:
            counterMineral = 0
            mineralNameList = [i for i,j in totalDict]
            mineralNameListPruned = [i.split(' ')[0] for i in mineralNameList]
            #mineralNameListDict = dict.fromkeys(set(mineralNameListPruned), False)
        
            normalized_spectraValue = preprocessing.normalize([testData[minIndexTemp:maxIndexTemp]])[0]
            
            minIndexTemp = 0
            maxIndexTemp = 0
            
            for indexRange in range(numberOfFeatureToAnalyze):
                minIndexTemp = minIndex[indexRange]
                maxIndexTemp = maxIndex[indexRange]
                
                baseline_values = peakutils.baseline(normalized_spectraValue)
                baselineRemoved_spectraValue = normalized_spectraValue - baseline_values
                
                #continumRemoved_spectraValue = sp.convex_hull_removal(normalized_spectraValue[minIndexTemp:maxIndexTemp], waveLength[minIndexTemp:maxIndexTemp])[0]
                xaxis = list(range(minWavelength[indexRange].value,maxWavelength[indexRange].value + 1))
                
                indexForWhile = 0
                counterMineral = 0
                mineralNameListDict = dict.fromkeys(set(mineralNameListPruned), False)
                while counterMineral < indidualplotlimit:
                    mineralName = totalDict[indexForWhile][0]
                    
                    if mineralNameListDict[mineralName.split(' ')[0]] == False:
                        fig = plt.figure(figsize=(24, 12))
                        plt.plot(xaxis, baselineRemoved_spectraValue, label = targetfilename.replace('.xlsx',''))
                        
                        
                        imagePath = list(s for s in txt_files if mineralName.lower() in s.lower())[0]
                        signal = globals()['fileListBaselineRemoval_'+str(indexRange)][txt_files.index(imagePath)]
                        plt.plot(xaxis, signal, label = mineralName)
                        mineralNameListDict[mineralName.split(' ')[0]] = True
                        counterMineral = counterMineral + 1
                        
                        targertFolderImages = esp_files[index].replace(targetFolderToReplace, targetFolderNewName).replace('.txt','') + '/'
                        
                        os.makedirs(targertFolderImages, exist_ok=True)
                        imageLabelRange = '_' + str(minIndexTemp + 160) + '_' + str(maxIndexTemp + 160 - 1)
                        individualImagePath = targertFolderImages + str(counterMineral) + '_' + mineralName.split('_')[0] + imageLabelRange + '.png'
                        
                        plt.legend(loc="upper left")
                        plt.savefig(individualImagePath, dpi=100)
                        plt.close()
                    
                    indexForWhile = indexForWhile + 1
        except:
            continue
            #raise
            
        
        if toggleButtonFullPlot.value == 'Yes':
            try:
                counterMineral = 0
                mineralNameList = [i for i,j in totalDict]
                mineralNameListPruned = [i.split(' ')[0] for i in mineralNameList]
            
                normalized_spectraValue = preprocessing.normalize([testData])[0]
                #continumRemoved_spectraValue = sp.convex_hull_removal(normalized_spectraValue, waveLength)[0]
                
                baseline_values = peakutils.baseline(normalized_spectraValue)
                baselineRemoved_spectraValue = normalized_spectraValue - baseline_values
                
                
                for indexRange in range(numberOfFeatureToAnalyze):
                    
                    indexForWhile = 0
                    counterMineral = 0
                    mineralNameListDict = dict.fromkeys(set(mineralNameListPruned), False)
                    while counterMineral < indidualplotlimit:
                        mineralName = totalDict[indexForWhile][0]
                        
                        if mineralNameListDict[mineralName.split(' ')[0]] == False:
                            fig = plt.figure(figsize=(24, 12))
                            plt.plot(waveLength, baselineRemoved_spectraValue, label = targetfilename.replace('.xlsx',''))
                            
                            
                            imagePath = list(s for s in txt_files if mineralName.lower() in s.lower())[0]
                            signal = fileListBaselineRemoval[txt_files.index(imagePath)]
                            plt.plot(waveLength, signal, label = mineralName)
                            mineralNameListDict[mineralName.split(' ')[0]] = True
                            counterMineral = counterMineral + 1
                            
                            targertFolderImages = esp_files[index].replace(targetFolderToReplace, targetFolderNewName).replace('.txt','') + '/fullWavelength/'
                            
                            os.makedirs(targertFolderImages, exist_ok=True)
                            individualImagePath =  targertFolderImages + str(counterMineral) + '_' + mineralName.split('_')[0] + '.png'
                            
                            plt.legend(loc="upper left") 
                            plt.savefig(individualImagePath, dpi=100)
                            plt.close()
                        
                        indexForWhile = indexForWhile + 1
            except:
                continue
             
            
    progreesBarEsp.style = {'description_width': 'initial'}
    progreesBarEsp.description = 'Processing files ' + str(index + 1)
    progreesBarEsp.value += 1 # signal to increment the progress bar
            

IntProgress(value=0, max=281)